<a href="https://colab.research.google.com/github/jamg-upv/0Reto21d_ago24_LLMclassification/blob/main/py/Reto21dias_24_preprocesamiento_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#conectar con repositorio github publico para acceso a los datasets
!git clone https://github.com/jamg-upv/0Reto21d_ago24_LLMclassification.git

Cloning into '0Reto21d_ago24_LLMclassification'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 56 (delta 23), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (56/56), 1.55 MiB | 3.13 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [ ]:
#conecar a google drive para guardar alli los outputs que quiera mantener (y que no se me olvide descargarlos) luego los subiré a Git para su uso posterior
from google.colab import drive
drive.mount('/content/drive')

In [6]:
import pandas as pd

# Leer el archivo CSV (es un CSV de Open Office si lo genero con MSexcel no se mapea bien sin dar atributos adicionales)
hiwp_data = pd.read_csv("/content/0Reto21d_ago24_LLMclassification/datasets/hiwp_componentsv2.csv")

# Seleccionar las filas de la 1 a la 12 (índices 0 a 11) que tienen las definiciones de componentes de HIWP. Comentar si quiero trabajar con filas adicionales
# limito explicitamente a las dos primeras columnas que son los datos con los que quiero trabajar (la tercera tiene comentarios de clasificacion de las filas articulos)
hiwp_data = hiwp_data.iloc[:, 0:2]

# Mostrar el DataFrame resultante
print(hiwp_data)

             Component                                        Description
0   Select&Recruitment  Rigorous Selection and Recruitment Selecting t...
1             Training  Continuous Training and Development Investing ...
2          Empowerment  Employee participation Autonomy and Empowermen...
3          PerformEval  Performance Evaluation and Feedback Providing ...
4        Compensations  Competitive Compensation and Benefits Offering...
5              FairJob  Fair Work Practices Treating employees fairly ...
6        Communication  Open and Transparent Communication Sharing inf...
7               Career  Career and Succession Planning Helping employe...
8        LeaderDevelop  Leadership Development A conscious effort to d...
9      WorkLifeBalance  Work-Life Balance Implementing policies and pr...
10         Recognition  Performance Recognition Recognizing and reward...
11     PositiveCulture  Positive Organizational Culture Fostering an o...
12     LongDescripHIWP  High Involveme

# Promtp en claude3.5

Tengo un dataframe de nombre ""hiwp_data"" donde la primera columna es una etiqueta del texto y la segunda columan es el texto que quiero procesar con embeedings mas adelante. Pero primero quiero hacer un preprocesamiento del texto para limpiarlo y dejarlo mejor preparado para las tareas posteriores (calcula la distancia de cada fragmento de texto sobre otros fragmentos de texto). Quiero poder hacer una o varias de estas tareas """1. Lowers all characters
2. Removes all extra white spaces if there is one
3. Removes all special characters including numbers and punctuations
4. Stopword Removal. Eliminating common, non-content words that don't carry much meaning and can clutter the data. You would like to extend this list.
5. Removing words less then 3 characters. You may want to keep some key acronyms before applying this step
6. Stemming/Lemmatization. Reducing words to their base form (stem) or dictionary form (lemma)
7. Bigrams and Trigrams. Building a sequence of two (bigrams) or three (trigrams) consecutive words. You can also make your own bigrams and trigrams manually.
8. Tokenisation. Breaks down text data into smaller, more manageable units  called tokens.""" ¿en qué orden me aconsejas realizarlas? ¿puedes prepararme fragmentos de codigo que realicen cada una de esas operaciones en python apra correr en google colab?

Te aconsejaría realizar las operaciones de preprocesamiento en el siguiente orden:

Convertir a minúsculas
Eliminar espacios en blanco extra
Eliminar caracteres especiales, números y puntuación
Tokenización
Eliminar stopwords
Eliminar palabras de menos de 3 caracteres
Lematización (o stemming)
Creación de bigramas y trigramas
Este orden permite una limpieza progresiva del texto, comenzando con operaciones más generales y avanzando hacia las más específicas. La tokenización se realiza antes de eliminar stopwords y palabras cortas para facilitar estas operaciones. La lematización se hace después de eliminar palabras cortas para no procesar innecesariamente tokens que luego serán eliminados. Finalmente, se crean los bigramas y trigramas con el texto ya limpio y procesado.

In [2]:
# # Instalar las dependencias necesarias
import pkg_resources
import subprocess

required_packages = [
    'nltk ',
    'pandas'
]

installed_packages = {pkg.key for pkg in pkg_resources.working_set}

for package in required_packages:
    if package in installed_packages:
        print(f"{package} ya está instalado.")
    else:
        print(f"{package} no está instalado. Instalando...")
        subprocess.check_call(["pip", "install", package])
        print(f"{package} ha sido instalado.")

print("\nTodas las bibliotecas necesarias están ahora instaladas.")

nltk  no está instalado. Instalando...
nltk  ha sido instalado.
pandas ya está instalado.

Todas las bibliotecas necesarias están ahora instaladas.


In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams

# Descargar recursos necesarios de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Cargar el DataFrame (asumiendo que ya está cargado como hiwp_data)
# hiwp_data = pd.read_csv('tu_archivo.csv')  # Descomenta esta línea si necesitas cargar el archivo

# Función para preprocesar el texto
def preprocess_text(text):
    # 1. Convertir a minúsculas
    text = text.lower()

    # 2. Eliminar espacios en blanco extra
    text = re.sub(r'\s+', ' ', text).strip()

    # 3. Eliminar caracteres especiales, números y puntuación
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # 4. Tokenización
    tokens = word_tokenize(text)

    # 5. Eliminar stopwords
    stop_words = set(stopwords.words('english'))  # Cambia 'spanish' por 'english' si el texto está en inglés
    tokens = [token for token in tokens if token not in stop_words]

    # 6. Eliminar palabras de menos de 3 caracteres
    tokens = [token for token in tokens if len(token) >= 3]

    # 7. Lematización
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # 8. Crear bigramas y trigramas
    bigrams = list(ngrams(tokens, 2))
    trigrams = list(ngrams(tokens, 3))

    # Unir tokens, bigramas y trigramas
    processed_text = ' '.join(tokens + [f'{b[0]}_{b[1]}' for b in bigrams] + [f'{t[0]}_{t[1]}_{t[2]}' for t in trigrams])

    return processed_text

# Aplicar el preprocesamiento a la columna de texto
hiwp_data['processed_text'] = hiwp_data['Description'].apply(preprocess_text)

# Mostrar las primeras filas del DataFrame procesado
print(hiwp_data.head())